In [264]:
import torch
import pandas as pd
import numpy as np

# read the csv data out
data = pd.read_csv("../Data/ProductData.csv")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)





In [265]:
# dictionary mapping a label to a number
cat_to_vec = {'Sports & Outdoors': 1, 'Clothing, Shoes & Jewelry': 2, 'Home & Kitchen': 3, 'Baby Products': 4, 'Pet Supplies': 5, 'Arts, Crafts & Sewing': 6, 'Electronics': 7, 'Video Games': 8, 'Patio, Lawn & Garden': 9, 'Tools & Home Improvement': 10, 'Office Products': 11, 'Grocery & Gourmet Food': 12, 'Remote & App Controlled Vehicle Parts': 13, 'Health & Household': 14, 'Industrial & Scientific': 15, 'Beauty & Personal Care': 16, 'Hobbies': 17, 'Remote & App Controlled Vehicles & Parts': 18, 'Automotive': 19, 'Musical Instruments': 20, 'Movies & TV': 21, 'Cell Phones & Accessories': 22, 'Toys & Games': 23, 'Appliances': 24, 'Books': 25, 'Mobility & Daily Living Aids': 26, 'Power & Hand Tools': 27, 'Medical Supplies & Equipment': 28, 'Kitchen & Dining': 29, 'Motorcycle & Powersports': 30, 'CDs & Vinyl': 31, 'Small Appliance Parts & Accessories': 32, 'Office Electronics': 33, 'Instrument Accessories': 34}


In [266]:
data.replace(to_replace=cat_to_vec, inplace=True)

In [267]:
data.head(10)

,Unnamed: 0,Product Name,Category
0,0,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",1
1,6,Rubie's Child's Pokemon Deluxe Pikachu Costume...,2
2,8,"ARTSCAPE Etched Glass 24"" x 36"" Window Film, 2...",3
3,12,Baby Doll Bedding Chevron Window Valance and C...,4
4,17,Flash Furniture 25''W x 45''L Trapezoid Red HP...,3
5,27,Industro Stainless Steel Hose Clamps,3
6,28,"Oopsy Daisy Birchwood Owl Growth Chart, Pink, ...",4
7,30,Franklin Sports Eye Black Stickers for Kids - ...,1
8,32,Jay Franco Disney Frozen 2 Forest Spirit Twin/...,3
9,34,"Forum Novelties Union Officer Child's Costume,...",2


In [268]:
#create vocab

# create an array of product name values
make_vocab = list(data[['Category', 'Product Name']].itertuples(index=False, name=None))
# make the array an iterable
train_iter = iter(make_vocab)

In [269]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [270]:
vocab(['jacket', 'sweater', 'napkin', 'bamboo'])

[1210, 2991, 1688, 678]

In [271]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [272]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


# train_iter may need to be rewritten so its not an iterable, ie list(data[['Category', 'Product Name']].itertuples(index=False, name=None))
train_iter = list(data[['Category', 'Product Name']].itertuples(index=False, name=None))
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [273]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, 50)
        self.activation = torch.nn.ReLU()
        self.sl = nn.Linear(50, num_class)
        self.softmax = torch.nn.Softmax(dim = 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.fc(embedded)
        x = self.activation(x)
        x = self.sl(x)
        x = self.softmax(x)
        return x

In [274]:
# again might need to rewrite train iter to not be an iterable
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
print(vocab_size, num_class)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

26999 34


In [275]:

import torch
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [276]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR =  0.1 # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = list(data[['Category', 'Product Name']].itertuples(index=False, name=None))[:10000]
test_iter = list(data[['Category', 'Product Name']].itertuples(index=False, name=None))[10000:]
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  1.22s | valid accuracy    0.230 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  1.35s | valid accuracy    0.230 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  1.33s | valid accuracy    0.230 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  1.22s | valid accuracy    0.230 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  1.15s | valid accuracy    0.230 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.99s |

In [53]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.310


C:\Users\hemph\AppData\Local\Temp\ipykernel_384\4169281971.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [90]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

external_text = "Franklin Sports Eye Black Stickers for Kids - Customizable Lettering Baseball and Football Eye Black Stickers"



In [91]:
vec_to_cat = {v: k for k, v in cat_to_vec.items()}

In [92]:
vec_to_cat[predict(external_text, text_pipeline)]

'Sports & Outdoors'